# Objective

Discover the data about truck vehicles with following fields:

<table align='left'>
    <tr>
        <th>Fields</th>
        <th>Url Search</th>
        <th>Url Scraping</th>
        <th></th>
        <th></th>
    </tr>
    <tr>
        <td>Plate</td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
    </tr>
    <tr>
        <td>Brand</td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
    </tr>
    <tr>
        <td>Model</td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
    </tr>
    <tr>
        <td>Year</td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
    </tr>
    <tr>
        <td>Chassi</td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
    </tr>
    <tr>
        <td>Eixos</td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
    </tr>
    <tr>
        <td>Traction</td>
        <td></td>
        <td></td>
        <td></td>
        <td></td>
    </tr>

</table>

# Qualveiculo Scraping

In [2]:
from requests import get
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs4
from time import sleep
from datetime import datetime

In [3]:
#    list_proxy = proxies
#
#    PROXY = list_proxy[random.randrange(len(list_proxy)-1)]
#    webdriver.DesiredCapabilities.FIREFOX['proxy'] = {
#       "httpProxy": PROXY,
#        "httpsProxy": PROXY,
#        "ftpProxy": PROXY,
#        "sslProxy": PROXY,
#        "proxyType": "MANUAL",
#    }


In [4]:
def scraping(url, plate, proxies, cont):

    plate = plate
    url = url
    proxies = proxies

    profile = webdriver.FirefoxProfile()
    profile.set_preference("network.proxy.type", 1)
    profile.set_preference("network.proxy.http", proxies[cont]['ip'])
    profile.set_preference("network.proxy.http_port", proxies[cont]['port'])
    profile.set_preference("network.proxy.https", proxies[cont]['ip'])
    profile.set_preference("network.proxy.https_port", proxies[cont]['port'])

    browser = webdriver.Firefox(profile)
    browser.get(url)
    sleep(6)

    xpath_field = 'placa'
    xpath_botton = 'consultar'

    elem = browser.find_element_by_name(xpath_field)
    elem.clear()
    elem.send_keys(plate)
    elem.send_keys(Keys.RETURN)
    sleep(5)

    soup = bs4(browser.page_source, 'html.parser')
    browser.quit()

    return soup

In [5]:
def data_transform(soup):

    tag = 'div'
    attribute = {'id':'resultado'}

    try:
        element = soup.find(tag, attribute).get_text().split('\n')
    except(TypeError, IndexError, AttributeError):
        element = None
    
    try:
        vazio = element[0]
    except(TypeError, IndexError, AttributeError):
        vazio = None
    try:
        dados = element[1]
    except(TypeError, IndexError, AttributeError):
        dados = None
    try:
        dados_texto = element[2]
    except(TypeError, IndexError, AttributeError):
        dados_texto = None
    try:
        dados_localidade = element[3]
    except(TypeError, IndexError, AttributeError):
        dados_localidade = None
    
    return vazio, dados, dados_texto, dados_localidade

In [6]:
def export_data(df_list):
    
    time_stampe = (str(datetime.now()).split('.')[0].replace('-',
                '_').replace(' ', '_').replace(':', '')[:-2])
    
    #----create dictionary to output return
    df_dict = {
    'plate':df_list[0], 'vazio':df_list[1], 'dados':df_list[2],
    'dados_texto':df_list[3], 'dados_localidade':df_list[4]
    }

    df = pd.DataFrame(df_dict)
    df.to_csv(f'data_col_{time_stampe}', index=False)
    
    return print(f'Sucess Exported_{time_stampe}')

In [7]:
def loop_plate():
    url = 'https://www.qualveiculo.net/'
    plates = pd.read_csv('PLACAS_2.csv', names=['plate'])
    plate, vazio, dados, dados_texto, dados_localidade = [],[],[],[],[]
    cont = 0

    # --------------------------------get each plate from list of plates
    for plate_item in plates.plate[:50]:
        soup = scraping(url, plate_item, proxies, cont)
        data = data_transform(soup)
        print(cont, plate_item, data)
        cont += 1

        #------------------------------append list for exportation to csv
        plate.append(plate_item), vazio.append(data[0]), dados.append(data[1])
        dados_texto.append(data[2]), dados_localidade.append(data[3])

    return plate, vazio, dados, dados_texto, dados_localidade

    #----data analysis
    #----GM = General Motors  "how is the GM in the database from unus? GM, General...Chevrolet, etc"
    #----SR, guerra > sr = 'semi reboque', guerra = 'brand', when has the SR in the brand, 
         # the first word in the model is the brand
    #----VW = Volkswagen


In [7]:
from proxy_generator import main
proxies = main()

ValueError: 1 is not in list

In [ ]:
export_data(loop_plate())

In [8]:
proxies

NameError: name 'proxies' is not defined